# Summary
We map cropland across Africa.
1. Using Segment Anything

### Future
-

# Set up the environment

In [ ]:
# Import and/or install libraries
import subprocess, os, gcsfs, json

try:
    import geemap, ee
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'geemap'])
    import geemap, ee

try:
    import samgeo
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', '-U', 'segment-geospatial'])
    from samgeo import SamGeo

!pip install localtileserver
!pip install pycrs

In [ ]:
# Connect to Google Drive to access files

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connect to Google Earth Engine if neccessary

service_account = os.environ.get('GOOGLE_SERVICE_ACCOUNT')
credentials = ee.ServiceAccountCredentials(service_account, os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))
ee.Initialize(credentials)

## Import AOI

In [ ]:
aoi = ee.Geometry({
        "type": "Polygon",
        "coordinates": [
          [
            [
              22.763716590708043,
              5.70609319915458
            ],
            [
              22.763716590708043,
              5.664906631466579
            ],
            [
              22.81810892203191,
              5.664906631466579
            ],
            [
              22.81810892203191,
              5.70609319915458
            ],
            [
              22.763716590708043,
              5.70609319915458
            ]
          ]
        ],
      })

# Create a cloudless Sentinel-2 collection

In [ ]:
CLOUD_FILTER = 80
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50


def get_s2_sr_cld_col(datefilter):
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filter(datefilter)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filter(datefilter)
        )

    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


def add_cloud_bands(img):
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    return img.addBands(ee.Image([cld_prb, is_cloud]))


def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)


def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [ ]:
date_filter = ee.Filter.date(ee.Date('2013-12-31'), ee.Date('2023-01-01'))

sentinel = get_s2_sr_cld_col(date_filter).map(add_cld_shdw_mask).map(apply_cld_shdw_mask)

sentinel_vis = {
  'min': 0.0,
  'max': 3000,
  'bands': ['B4', 'B3', 'B2'],
  'gamma': 1.4
}

sentinel_clip = sentinel.filterBounds(aoi).median()

In [ ]:
image_name = 'sentinel_bakouma'
fileNamePrefix = 'temp/' + image_name

exportConfig = {
    'image': sentinel_clip.visualize(**sentinel_vis),
    'description': image_name,
    'bucket': 'nature-watch-bucket',
    'fileNamePrefix': fileNamePrefix,
    'scale': 10,
    'maxPixels': 3147395000,
    'region': aoi,
    'fileFormat': 'GeoTIFF',
    'formatOptions': {'cloudOptimized': True}
}

task = ee.batch.Export.image.toCloudStorage(**exportConfig)
task.start()

In [ ]:
ee.data.listOperations()

# Segmentation

In [ ]:
import os
import numpy as np
import rasterio

# Define the Google Cloud Storage bucket and GeoTIFF file.
bucket = 'nature-watch-bucket'
filename = 'temp/sentinel_bakouma.tif'
url = f'https://storage.googleapis.com/{bucket}/{filename}'

# Set the GDAL configuration options.
rasterio.env.GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR'
rasterio.env.CPL_VSIL_CURL_ALLOWED_EXTENSIONS='.tif'

# Read the GeoTIFF file from Google Cloud Storage.
with rasterio.open(f'/vsicurl/{url}') as src:
    image = src.read()
    transform = src.transform
    crs = src.crs

# Convert the image to a numpy array.
array = np.array(image)
array = np.transpose(array, (1, 2, 0))


In [ ]:
print(array.shape)

In [ ]:
sam_kwargs = {
    "points_per_side": 32,
    "pred_iou_thresh": 0.86,
    "stability_score_thresh": 0.92,
    "crop_n_layers": 1,
    "crop_n_points_downscale_factor": 2,
    "min_mask_region_area": 10,
}

sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    device=None,
    sam_kwargs=None,
)

In [ ]:
sam.generate(array, output="masks.tif", foreground=False, unique=True)
sam.show_masks(cmap="binary_r")

In [ ]:
sam.show_anns(axis="off", alpha=1, output="annotations.tif")

In [ ]:
sam.tiff_to_vector("masks.tif", "masks.shp")

# Map

In [ ]:
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.addLayer(sentinel_clip, sentinel_vis, 'sentinel_clip')
Map.add_vector("masks.shp", alpha=0.5, layer_name="Masks")
Map.setCenter(31.866, -25.358, 12)
Map